In [ ]:
!mamba install mkl=2019.1.* -c pytorch -y -q

In [ ]:
!conda install pytorch=1.12 torchvision==0.13.1 torchaudio==0.12.1 cudatoolkit=11.3 -c pytorch -y -q

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html #==2.1.0+pt112cu113
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html #==0.6.16+pt112cu113
!pip install torch-geometric #==2.2.0

In [ ]:
!mamba install -c conda-forge pyts -q -y #==0.12.0

In [ ]:
!pip install llvmpy
!pip install cython#==0.29.24
!pip install numba#==0.56.2
!pip install pandas#==1.3.4
!pip install networkx#==2.6.3
!pip install matplotlib#==3.4.3
!pip install ts2vg#==1.0.0
!pip install pytorch_lightning==1.6.5
!pip install tensorflow#==2.11.0
!pip install dvclive#==1.3.3
# !pip install torchsummary
!pip install torchmetrics==0.6.0

# nyu

In [ ]:
!mamba install pytorch-cuda=11.6 -c pytorch -c conda-forge -c nvidia -y -q

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 -q

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.1+cu116.html -q

In [ ]:
!pip install ts2vg -q

In [ ]:
!pip install pytorch_lightning==1.6.5 -q

In [ ]:
!mamba install tensorflow-gpu -y -q

In [ ]:
!mamba install -c conda-forge pyts -q -y #==0.12.0

In [ ]:
!pip install torchsummary -q

# main

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import warnings
import sklearn
import os.path as osp
import matplotlib.pyplot as plt
import ts2vg
import pytorch_lightning as pl
import tensorflow as tf
import datetime

from pyts.image import MarkovTransitionField

from torch.nn import Linear, CrossEntropyLoss, BCEWithLogitsLoss
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool, ChebConv, global_sort_pool
from torch_geometric.loader import DataLoader
from torch.nn import Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv, GINEConv, GATv2Conv, GATConv

from tqdm import tqdm

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

from torchvision import transforms

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.callbacks import LearningRateFinder
#from pytorch_lightning.callbacks import BatchSizeFinder
from pytorch_lightning.callbacks import ProgressBarBase
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger

from ts2vg import NaturalVG
from ts2vg import HorizontalVG

In [ ]:
count = 0
for i in Y[2123*3:2123*4]:
    if i == 3:
        count += 1
print(count)

In [ ]:
        graph_type = "vis" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
        classif = "graph" #"graph", "node", set the type of classification
        len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

        path_main = "datasets/dataset_50_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
        path_properties = "datasets/dataset_properties.npz"  # path to properties used for random 
        path_mask = "datasets/dataset_mask.npz" # path to mask dataset used for random

        learning_rate = 0.01
        batch_size = 64
        range_epoch = 1000 #set length of epoch
        save_file="Models"
        name_of_save = "VG_node_classification_rounded"
        patience = 500
        analysis = True
        
        # df = pd.read_csv(path_main)  
        # del df['Unnamed: 0']
        # df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        # length_rss = int((df.columns.stop-2)/2)
        # global X_mask
        # X = df.loc[:,df.columns[:length_rss]].to_numpy()
        # #X = np.round_(X,1)
        # Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        # X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = torch.from_numpy(df.loc[:,df.columns[:length_rss]].values) # x values for every sample
        #X = np.round_(X,1)
        Y = torch.from_numpy(df[length_rss+1].values).type(torch.long) # types of anomalies
        X_mask = torch.from_numpy(df.loc[:,df.columns[length_rss+2:]].values) # binary location of anomalies
        

In [ ]:
W=[]
for i, val in enumerate(Z):
    if val == -87.63:
        print(i)
        W.append(val)
W

In [ ]:
K = np.zeros([50*5], dtype='float')
KM = np.zeros([50*5], dtype='float')
KA = np.zeros([50*5], dtype='float')

K[:50]=(X[(2123*0)+744])
K[200:]=(X[(2123*1)+1368])
K[50:100]=(X[(2123*2)+1699])
K[100:150]=(X[(2123*3)+1842])
K[150:200]=(X[(2123*4)+1864])

KM[:50]=(X[(2123*0)+744])
KM[200:]=(X[(2123*0)+1368])
KM[50:100]=(X[(2123*0)+1699])
KM[100:150]=(X[(2123*0)+1842])
KM[150:200]=(X[(2123*0)+1864])

KA[:50]=(X_mask[(2123*0)+744])
KA[200:]=(X_mask[(2123*1)+1368])
KA[50:100]=(X_mask[(2123*2)+1699])
KA[100:150]=(X_mask[(2123*3)+1842])
KA[150:200]=(X_mask[(2123*4)+1864])

In [ ]:
ji=0
temp=[]
for i in range(len(KA)):
    if ji != KA[i]:
        temp.append(i)
        if ji == 0:
            ji = 1
        else:
            ji =0
ji = temp
temp

In [ ]:
from matplotlib.pyplot import figure
Y_index= list(range(0, 250))
figure(dpi=200, figsize=(9, 4))
k = 0.5
g = 1
const = 'black'
plt.plot(KM, color=const, linewidth=0.3,linestyle='dotted')
plt.plot(Y_index[:ji[0]],K[:ji[0]], color=const, linewidth=k, label = 'No anomaly')
plt.plot(Y_index[ji[0]-1:ji[1]+1],K[ji[0]-1:ji[1]+1], color='red', linewidth=g, label='SuddenR anomaly')
plt.plot(Y_index[ji[1]:ji[2]],K[ji[1]:ji[2]], color=const, linewidth=k)
plt.plot(Y_index[ji[2]-1:ji[3]+1],K[ji[2]-1:ji[3]+1], color='blue', linewidth=g, label='InstaD anomaly')
plt.plot(Y_index[ji[3]:ji[4]],K[ji[3]:ji[4]], color=const, linewidth=k)
plt.plot(Y_index[ji[4]-1:ji[5]+1],K[ji[4]-1:ji[5]+1], color='blue', linewidth=g)
# plt.plot(Y_index[ji[5]:ji[6]],K[ji[5]:ji[6]], color=const, linewidth=k)
# plt.plot(Y_index[ji[6]-1:ji[7]+1],K[ji[6]-1:ji[7]+1], color='blue', linewidth=g)
# plt.plot(Y_index[ji[7]:ji[8]],K[ji[7]:ji[8]], color=const, linewidth=k)
# plt.plot(Y_index[ji[8]-1:ji[9]+1],K[ji[8]-1:ji[9]+1], color='blue', linewidth=g)
plt.plot(Y_index[ji[5]:ji[6]+1],K[ji[5]:ji[6]+1], color=const, linewidth=k)
plt.plot(Y_index[ji[6]-1:ji[7]+1],K[ji[6]-1:ji[7]+1], color='green', linewidth=g, label='SlowD anomaly')
plt.plot(Y_index[ji[7]:ji[8]+1],K[ji[7]:ji[8]+1], color=const, linewidth=k)
plt.plot(Y_index[ji[8]-1:],K[ji[8]-1:], color='orange', linewidth=g, label='SuddenD anomaly')

plt.xlabel('packet number (0.1 sec/packet)')
plt.ylabel('RSS (dBm)')

plt.legend()
plt.savefig('start_image.eps', format='eps')
plt.savefig('start_image.png', format='png')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.array([1, 2, 3, 4, 5, 6])
y = np.array([2, 4, 6, 8, 10, 12])
mask = np.array([1, 0, 1, 0, 1, 0])

mask_indices = np.where(mask == 1)[0]

plt.plot(x[mask_indices], y[mask_indices], 'ro')

mask_indices = np.where(mask == 0)[0]

plt.plot(x[mask_indices], y[mask_indices], 'bo')
plt.show()

In [ ]:
        k = 8015
        g = NaturalVG(weighted='abs_v_distance')
        #g = HorizontalVG(weighted='h_distance')
        
        g.build(X[k])

        adj_mat_vis = np.zeros([len(X[k]),len(X[k])], dtype='float')
        for i in range(len(g.edges)):
            x, y, q = g.edges[i]
            adj_mat_vis[x,y] = q
            #adj_mat_vis[y,x] = q
        
        edge_index = torch.from_numpy(adj_mat_vis).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat_vis[row, col]
        plt.imshow(adj_mat_vis)

In [ ]:
graph_type = "dual_VG" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
vis_type = "horizontal" #"natural", "horizontal"
vis_distance = 'abs_v_distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"

def vis_matrix(X_current, vis_type_local, dual = False):
    if vis_type_local == "natural":
        g = NaturalVG(weighted='abs_v_distance')
    elif vis_type_local == "horizontal":
        g = HorizontalVG(weighted='abs_v_distance')

    g.build(X_current)

    adj_mat_vis = np.zeros([len(X_current),len(X_current)], dtype='float')
    for i in range(len(g.edges)):
        x, y, q =g.edges[i]
        adj_mat_vis[x,y] = q
        if vis_edge_type == "undirected":
            adj_mat_vis[y,x] = q
    
    if dual == True and vis_edge_type == "directed":
        adj_mat_vis =  adj_mat_vis + vis_matrix(-X_current,"natural", vis_type).T
    
    return adj_mat_vis

pos_adj_mat_vis = vis_matrix(X[8001], "natural",dual = True)
# neg_adj_mat_vis = vis_matrix(-X[8001], "natural","directed")

edge_index = torch.from_numpy(pos_adj_mat_vis).nonzero().t().contiguous()

#join two edge_weight arrays
row, col = edge_index
# edge_weight = np.zeros([len(row),2], dtype='float')
edge_weight = pos_adj_mat_vis[row, col]
# edge_weight[:,1] = neg_adj_mat_vis[row, col]


In [ ]:
plt.imshow(pos_adj_mat_vis + neg_adj_mat_vis)

In [ ]:
edge_index

In [ ]:
plt.imshow(neg_adj_mat_vis)

In [ ]:
Dict = {

        "Normal" : 0,
        "Vertical" : 100000,  
        "AbsVertical" : 200000,  
        "MTF" : 0,  
        "VG" : 1000, 
        "MTF_on_VG" : 2000,  
        "VG_on_MTF" : 3000,  
        "HVG" : 4000,  
        "VG_dir" : 5000,
        "graph" : 0,  
        "node" : 100,  
        "uncut" : 0,  
        "random" : 10,  

        "50" : 20,  
        "100" : 30,  
        "150" : 40,  
        "200" : 50,  
        "250" : 60,  
        "25" : 70
}
Dict["Normal"] + Dict["VG"]

In [ ]:
X = (-71,-69,-90,-89,-67,-69,-68,-69)
#g = NaturalVG(weighted='distance')
g = HorizontalVG(weighted='distance', directed = None) # left_to_right # top_to_bottom

g.build(X)

adj_mat_vis = np.zeros([len(X),len(X)], dtype='float')
for i in range(len(g.edges)):
    x, y, q = g.edges[i]
    adj_mat_vis[x,y] = q
    adj_mat_vis[y,x] = q

edge_index = torch.from_numpy(adj_mat_vis).nonzero().t().contiguous()
row, col = edge_index
edge_weight = adj_mat_vis[row, col]
plt.imshow(adj_mat_vis)
plt.savefig('adj_matrix_plot.png', format='png')

In [ ]:
g.edges[1]

In [ ]:
g.weights

In [ ]:
edge_weight

In [ ]:
plt.imshow(adj_mat_vis)
plt.xlabel("Time steps")
plt.ylabel("RSSI")
# plt.savefig('adj_matrix_plot.eps', format='eps')
# plt.savefig('adj_matrix_plot.png', format='png')

In [ ]:
"""

in this bracket of code is located the conversion of time series to different graph neural networks (Markov transition field (MTF) and Visibility graphs (VG)) 
with different lengths (uncut(300), cut and random(where every sample has a different length between 300 and 50)).

create graph is called when we have a time series that needs to be converted into a graph. I

"""
def create_graph():
    warnings.filterwarnings("ignore")
    
    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy() # x values for every sample
        #X = np.round_(X,1)
        Y = df[length_rss+1].to_numpy(dtype=np.uint8) # types of anomalies
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy() # binary location of anomalies
        
        if graph_type in ("MTF", "vis_on_MTF", "MTF_on_vis"):
            MTF = MarkovTransitionField(n_bins=length_rss)
            X_gaf_MTF = MTF.fit_transform(X)
        
    # preparation for random graphs
    elif len_type == "random":
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            dataset_properties[i,1] = int(dataset_properties[i,1])
        
        X = dataset_rss # x values for every sample
        X_mask = dataset_mask # binary location of anomalies
        Y = dataset_properties[:,2] # types of anomalies
        Y_len = dataset_properties[:,0] # length of every sample
        
        if graph_type in ("MTF", "vis_on_MTF", "MTF_on_vis"):
            X_gaf_MTF = []
            for i in range(len(Y_len)):
                MTF = MarkovTransitionField(n_bins=Y_len[i])
                X_gaf_MTF_temp = MTF.fit_transform(X[i].reshape(1, -1))
                X_gaf_MTF.append(X_gaf_MTF_temp[0])
    
    # output will contain all graphs 
    output = []
    
    # setting class_weights for graph
    global class_weights
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(Y),
                                                                   y=Y))
    def vis_matrix(X_current, vis_type_local):
        if vis_type_local == "natural":
            g = NaturalVG(weighted=vis_distance)
        elif vis_type_local == "horizontal":
            g = HorizontalVG(weighted=vis_distance)
            
        g.build(X_current)

        adj_mat_vis = np.zeros([len(X_current),len(X_current)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_vis[x,y] = q
            if vis_edge_type == "undirected":
                adj_mat_vis[y,x] = q
        
        return adj_mat_vis
    
    # functions for creating edge index and edge weight for a given matrix
    def adjToEdgidx(adj_mat):
        #function for visibility and MTF matrixes
        edge_index = torch.from_numpy(adj_mat).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat[row, col]
        return edge_index,edge_weight
    
    def adjToEdgidx_join(adj_mat_MTF,X_current):
        #function for joined visibility and MTF matrixes
        adj_mat_vis = vis_matrix(X_current, vis_type)
            
        if graph_type == "vis_on_MTF":
            temp_main_adj_mat = adj_mat_MTF
        elif graph_type == "MTF_on_vis":
            temp_main_adj_mat = adj_mat_vis
            
        edge_index = torch.from_numpy(temp_main_adj_mat).nonzero().t().contiguous()
        
        #join two edge_weight arrays (visibility is converted to fit MTF) 
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = adj_mat_MTF[row, col]
        edge_weight[:,1] = adj_mat_vis[row, col]
        return edge_index, edge_weight
    
    def adjToEdgidx_dual_VG(X_current):
        #function for joined visibility and MTF matrixes
        pos_adj_mat_vis = vis_matrix(X_current, vis_type)
        neg_adj_mat_vis = vis_matrix(-X_current, vis_type)
            
        edge_index = torch.from_numpy(pos_adj_mat_vis+neg_adj_mat_vis).nonzero().t().contiguous()
        
        #join two edge_weight arrays
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = pos_adj_mat_vis[row, col]
        edge_weight[:,1] = neg_adj_mat_vis[row, col]
        return edge_index, edge_weight
    
    def adjToEdgidx_double_VG(X_current):
        
        adj_mat_vis1 = vis_matrix(X_current, "Natural")
        adj_mat_vis2 = vis_matrix(X_current, "Horizontal")
            
        edge_index = torch.from_numpy(adj_mat_vis1).nonzero().t().contiguous()
        
        #join two edge_weight arrays (visibility is converted to fit MTF) 
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = adj_mat_vis1[row, col]
        edge_weight[:,1] = adj_mat_vis2[row, col]
        
        return edge_index, edge_weight
    
    def define_mask(i):
        if classif == "graph": # for graph classification
            return torch.tensor(Y[i], dtype=torch.long)      
        elif classif == "node":# for node classification 
            return torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
        
    if graph_type == "MTF":  
        for i, j in enumerate(X_gaf_MTF):
            print(i)
            edge_index, edge_weight = adjToEdgidx(j)
            y_mask = define_mask(i)
            #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally label
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1), y=y_mask))
    
    elif graph_type == "vis":
        for i in range(len(X)):
            print(i)
            edge_index, edge_weight = adjToEdgidx(vis_matrix(X[i],vis_type))
            y_mask = define_mask(i)
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=torch.tensor(edge_index, dtype=torch.int64), edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1),y=y_mask))    
    
    elif graph_type in ("vis_on_MTF", "MTF_on_vis"):
        for i, j in enumerate(X_gaf_MTF):
            print(i)
            edge_index, edge_weight = adjToEdgidx_join(j, 
                                                       vis_matrix(X[i],vis_type))
            y_mask = define_mask(i) 
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.tensor(edge_weight, dtype=torch.double), y=y_mask))    
    elif graph_type == "double_VG":
        for i in range(len(X)):
            print(i)
            edge_index, edge_weight = adjToEdgidx_double_VG(X[i])
            y_mask = define_mask(i) 
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.tensor(edge_weight, dtype=torch.double), y=y_mask))    
    
    elif graph_type == "dual_VG":
        for i in range(len(X)):
            print(i)
            edge_index, edge_weight = adjToEdgidx_dual_VG(X[i])
            y_mask = define_mask(i) 
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.tensor(edge_weight, dtype=torch.double), y=y_mask))    
    
    return output    


In [ ]:
output = create_graph()

In [ ]:
#keeps the last output so we dont have to generate another if all parameters are the same as before
global temp_repeat
temp_repeat=['']*9
def generate_output():
    if temp_repeat[0] != graph_type or temp_repeat[1] != classif or temp_repeat[2] != len_type or temp_repeat[3] != path_main or temp_repeat[4] != path_properties or temp_repeat[5] != path_mask or temp_repeat[6] != vis_type or temp_repeat[7] != vis_distance or temp_repeat[8] != vis_edge_type:
    
        global output
        output = create_graph()
    temp_repeat[0] = graph_type 
    temp_repeat[1] = classif
    temp_repeat[2] = len_type
    temp_repeat[3] = path_main
    temp_repeat[4] = path_properties
    temp_repeat[5] = path_mask
    temp_repeat[6] = vis_type
    temp_repeat[7] = vis_distance
    temp_repeat[8] = vis_edge_type
    
    print(temp_repeat[1] + " classification on " + temp_repeat[2] + " time series using " + temp_repeat[0] + " graphs" )
    return output

def check_for_missclick():
    return graph_type in ("vis", "MTF", "MTF_on_vis", "vis_on_MTF","double_VG","dual_VG") and classif in ("graph", "node") and len_type in ("un/cut", "random") and vis_type in ("natural", "horizontal") and vis_distance in ('slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance') and vis_edge_type in ("undirected", "directed")
    

In [ ]:
# class FineTuneLearningRateFinder(LearningRateFinder):
#     def __init__(self, milestones, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.milestones = milestones

#     def on_fit_start(self, *args, **kwargs):
#         return

#     def on_train_epoch_start(self, trainer, pl_module):
#         if trainer.current_epoch in self.milestones or trainer.current_epoch == 0:
#             self.lr_find(trainer, pl_module)
            
# class LitProgressBar(ProgressBarBase):

#     def __init__(self):
#         super().__init__() 
#         self.enable = True

#     def disable(self):
#         self.enable = False

#     def on_train_batch_end(self, trainer, pl_module, outputs, batch_idx):
#         super().on_train_batch_end(trainer, pl_module, outputs, batch_idx)  # don't forget this :)
#         percent = (self.train_batch_idx / self.total_train_batches) * 100
#         sys.stdout.flush()
#         sys.stdout.write(f'{percent:.01f} percent complete \r')

# lr_finder = FineTuneLearningRateFinder(milestones=(5,10))
            
class GINE(pl.LightningModule):
    def __init__(self):
        super(GINE, self).__init__()
        
        if graph_type in ("MTF", "vis"):
            edge_dim = 1
        elif graph_type in ("MTF_on_vis", "vis_on_MTF","double_VG","dual_VG"):
            edge_dim = 2
            
        dim_h = 32
    
        self.conv1 = GINEConv(
            Sequential(Linear(dim_h, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv4 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv5 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        
        self.lin1 = Linear(dim_h*5, dim_h*5)
        self.lin2 = Linear(dim_h*5, 5)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_attr=edge_weight)
        h2 = self.conv2(h1, edge_index, edge_attr=edge_weight)
        h3 = self.conv3(h2, edge_index, edge_attr=edge_weight)
        h4 = self.conv4(h3, edge_index, edge_attr=edge_weight)
        h5 = self.conv5(h4, edge_index, edge_attr=edge_weight)
        
        # Graph-level readout
        
        h1 = global_max_pool(h1, batch)
        h2 = global_max_pool(h2, batch)
        h3 = global_max_pool(h3, batch)
        h4 = global_max_pool(h4, batch)
        h5 = global_max_pool(h5, batch)
        
        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3, h4, h5), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
                     
        out = model(train_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device) #weight=class_weights
        train_loss = loss_function(out, train_batch.y)
        
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        val_loss = loss_function(out, val_batch.y)
        
        pred = out.argmax(-1)
        correct=out.argmax(dim=1).eq(val_batch.y).sum().item()
        total=len(val_batch.y)
        val_label = val_batch.y
        accuracy = (pred == val_label).sum() / pred.shape[0]
        
        logs={"train_loss": val_loss}
        batch_dictionary={"loss": val_loss, "log": logs, "correct": correct, "total": total}
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
        
    
    def test_step(self, test_batch, batch_idx):
        # this is the test loop
        out = model(test_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        test_loss = loss_function(out, test_batch.y)
        
        pred = out.argmax(-1)
        test_label = test_batch.y
        accuracy = (pred == test_label).sum() / pred.shape[0]
        self.log("test_true", test_label)
        self.log("test_pred", pred)
        self.log("test_acc", accuracy)
        return pred, test_label
        
    def test_epoch_end(self, outputs):
        true_array=[]
        pred_array = []
        for i in range(len(outputs)):
            true_array = np.append(true_array,outputs[i][1].cpu().numpy())
            pred_array = np.append(pred_array,outputs[i][0].cpu().numpy())            
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array, digits=3))
    
#---------------------------------------------------------------------------------
class Net(pl.LightningModule):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = GATConv(1, 32, heads=4)
        self.lin1 = torch.nn.Linear(1, 4 * 32)
        self.conv2 = GATConv(4 * 32, 32, heads=4)
        self.lin2 = torch.nn.Linear(4 * 32, 4 * 32)
        self.conv3 = GATConv(4 * 32, 1, heads=6,concat=False)
        self.lin3 = torch.nn.Linear(4 * 32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    


    def training_step(self, train_batch, batch_idx):        
        out = model(train_batch)
        loss_function = BCEWithLogitsLoss().to(device) #weight=class_weights[1]
        
        train_loss = loss_function(out, train_batch.y)
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        val_loss = loss_function(out, val_batch.y)
        
        ys, preds = [], []
        val_label = val_batch.y.cpu()
        ys.append(val_batch.y)
        preds.append((out > 0).float().cpu())     
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == val_label).sum() / pred.shape[0]
    
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
    
    def test_step(self, test_batch, batch_idx):
        # this is the test loop
        out = model(test_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        test_loss = loss_function(out, test_batch.y)
        
        ys, preds = [], []
        test_label = test_batch.y.cpu()
        ys.append(test_batch.y)
        preds.append((out > 0).float().cpu())
        
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == test_label).sum() / pred.shape[0]
        
        self.log("test_acc", accuracy)
        return pred, y
        
    def test_epoch_end(self, outputs):
        global true_array, pred_array
        true_array=[]
        pred_array = []
        for i in range(len(outputs)):
            true_array = np.append(true_array,outputs[i][1].cpu().numpy())
            pred_array = np.append(pred_array,outputs[i][0].cpu().numpy())   
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array, digits=3))
        print("pred_array ",pred_array)
        pd.DataFrame(pred_array).to_csv("pred_array.csv")
        pd.DataFrame(true_array).to_csv("true_array.csv")

        
def main():
    if check_for_missclick() != True:
        print("one of the main parameters does not match")
    else:
        global model, test_loader, device
        
        early_stop = EarlyStopping(monitor='val_acc',patience=150, strict=False,verbose=False, mode='max')
        model_chackpoint = ModelCheckpoint(monitor='val_loss', mode='min')
        val_checkpoint_acc = ModelCheckpoint(filename="max_acc-{epoch}-{step}-{val_acc:.3f}", monitor = "val_acc", mode="max")
        val_checkpoint_best = ModelCheckpoint(filename="best", monitor = "val_acc", mode="max")
        val_checkpoint_loss = ModelCheckpoint(filename="min_loss-{epoch}-{step}-{val_loss:.3f}", monitor = "val_loss", mode="min")
        latest_checkpoint = ModelCheckpoint(filename="latest-{epoch}-{step}", monitor = "step", mode="max",every_n_train_steps = 500,save_top_k = 1)
        batchsizefinder = BatchSizeFinder(mode='power', steps_per_trial=3, init_val=2, max_trials=25, batch_arg_name='batch_size')
        
        torch.manual_seed(SEED)
        print(SEED)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        output = generate_output()
        # my data        
        
        train_size = int(0.8 * len(output))
        Temp_size = len(output) - train_size
        val_size = int(0.2*Temp_size)
        test_size = Temp_size - val_size
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])
                
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        # mode
        if classif == "graph":
            model = GINE().double()#.to(device) #GINE is for graph classification
        elif classif == "node":
            model = Net().double()#.to(device) #net is for node classification
            
        #training
        logger = TensorBoardLogger(save_file, name=name_of_save) # where the model saves the callbacks
        val_check_interval=len(train_loader)
        bar = LitProgressBar()
        
        trainer = pl.Trainer(logger=logger,max_epochs = range_epoch, callbacks=[latest_checkpoint, val_checkpoint_acc,val_checkpoint_loss,val_checkpoint_best,early_stop],accelerator='gpu',devices=1)
        trainer.fit(model, train_loader, val_loader)
        
def result_showcase():
    if check_for_missclick() != True:
        print("one of the main parameters does not match")
    else:
        global model, test_loader, device
    
        import os, sys
        versions = os.listdir(name_of_save + "/"+version)
        len_ver = len(versions)
        SEED_temp=SEED
        for i in range(len_ver):#len_ver

            global device, model, train_loader,val_loader, test_loader

            torch.manual_seed(SEED_temp)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            output = generate_output()
            train_size = int(0.8 * len(output))
            Temp_size = len(output) - train_size
            val_size = int(0.2*Temp_size)
            test_size = Temp_size - val_size
            train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
                        # mode
            if classif == "graph":
                model = GINE().load_from_checkpoint(name_of_save + "/"+version+"/version_" + str(i) + "/checkpoints/" + monitor_test).double() # "DvcLiveLogger/dvclive_run/checkpoints"+ monitor_test
            elif classif == "node":
                model = Net.load_from_checkpoint(name_of_save+"/"+version+"/version_" + str(i) + "/checkpoints/" + monitor_test).double()
            trainer = pl.Trainer(accelerator='gpu',devices=1)
            trainer.test(model, test_loader)
            print("version_"+str(i))
            print(SEED_temp)
            SEED_temp += 1

#------------------------------------------------------------


#main parametrs

graph_type = "MTF" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "random" #"un/cut", "random", the shape of data used in later paths 
#paths
path_main = "datasets/dataset_rss.npz" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask.npz" # path to mask dataset used for random

# params for 
SEED = 10
monitor_test = "best_acc.ckpt"
learning_rate = 0.01
batch_size = 64*2
range_epoch = 500 #set length of epoch
save_file="tb_logs"
name_of_save = "test"
analysis = True
#-------------------------------------------------------------

In [ ]:
version = "vis_1206000"
monitor_test = "best_acc.ckpt"

graph_type = "vis" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
vis_type = "natural" #"natural", "horizontal"
vis_distance = 'abs_v_distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"

path_main = "datasets/dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties_50_300.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask_50_300.npz" # path to mask dataset used for random
SEED = 205000

result_showcase()

In [ ]:
version = "HVG_dual_v_abs_209000"
monitor_test = "best_acc.ckpt"

graph_type = "dual_VG" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
vis_type = "horizontal" #"natural", "horizontal"
vis_distance = 'abs_v_distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"


In [ ]:
version = "HVG_dual_v_abs_209000"
monitor_test = "best_acc.ckpt"

graph_type = "dual_VG" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
vis_type = "horizontal" #"natural", "horizontal"
vis_distance = 'abs_v_distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"

path_main = "datasets/dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties_50_300.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask_50_300.npz" # path to mask dataset used for random
SEED = 209000

result_showcase()

In [ ]:
def anomaly_type(my_model):
    if len(my_model) == 300:
        my_model[0] = 0
        my_model[1] = 0
        my_model[2] = 0

    if len(my_model)%2 != 0
        my_model = np.append(0,my_model)
    par1 = int(len(my_model)/2)
    
    #check if anywhere in array there is an anomaly, otherwise predict this is a No_anomaly
    if any(my_model) == 1:
        
        mm1 = [all(i) for i in my_model.reshape(par1, -1)]
        mm2 = [all(i) for i in my_model[1:-1].reshape(par1, -1)]
        #check if anywhere in the array there are two 1 gruped together, otherwise predict this is a InstaD
        if any(mm1) == 1 and any(mm2) == 1:
            
            #check if last in the array is 1 (this tells me if the anomaly ends or not), otherwise predict this is a SuddenR
            if my_model[-1] == 1:
                
                mm3 = [all(i) for i in my_model.reshape(2, -1)]
                #you can't differentiate SlowD from SuddenD other than that that SlowD starts very early in the array, so if second half of the array is all 1 predict Slow, otherwise predict this is a SuddenD
                if any(mm3) == 1:
                    
                    return "SlowD"
                return "SuddenD"
            return "SuddenR"
        return "InstaD"
    return "No_anomaly"

In [ ]:
def Anomaly_loc(pred_type,pred_pred):
    W_list = []
    for i in range(len(pred_type)):
        
        result = [j for j, x in enumerate(pred_model[i]) if x]
        
        if pred_type[i] == "No_anomaly":
            W = "There is no anomaly"
            
        elif pred_type[i] == "InstaD":
            result_str = [str(a) for a in result]
            W = "Anomaly type is Instant Degredation and the anomalies are located at times "+', '.join(result_str)
            
        elif pred_type[i] == "SuddenR":
            W = "Anomaly type is Sudden Recovery and the anomalies are located at times from " + str(result[0]) + " to " + str (result[-1])
            
        elif pred_type[i] == "SuddenD":
            W = "Anomaly type is Sudden Degredation and the anomalies are located at times from " + str(result[0]) + " and do not stop"
        
        elif pred_type[i] == "SlowD":
            W = "Anomaly type is Slow Degredation and the anomalies are located at times from " + str(result[0]) + " and do not stop"
        W_list = np.append(W_list, W)

    return W_list

In [ ]:
pred_model = pd.read_csv("datasets/pred_array.csv") 
true_model = pd.read_csv("datasets/true_array.csv")
del pred_model['Unnamed: 0']
del true_model['Unnamed: 0']
pred_model = np.array(pred_model).reshape(1699, -1)
true_model = np.array(true_model).reshape(1699, -1)

# pred_array2 = np.array([])
# for i in pred_model:
#     pred_array2 = np.append(pred_array2, anomaly_type(i))

# true_array2 = np.array([])
# for i in true_model:
#     true_array2 = np.append(true_array2, anomaly_type(i))
    
# Tell_me = Anomaly_loc(pred_array2, pred_model)

In [ ]:
i=9
print(pred_model[i])
print(true_model[i])

In [ ]:
Tell_me

In [ ]:
        dataset_rss = np.load("datasets/dataset_rss_50_300.npz", allow_pickle=True)['arr_0']
        dataset_properties = np.load("datasets/dataset_properties_50_300.npz", allow_pickle=True)['arr_0']
        dataset_mask = np.load("datasets/dataset_mask_50_300.npz", allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            dataset_properties[i,1] = int(dataset_properties[i,1])
        
        X = dataset_rss
        X_mask = dataset_mask
        Y = dataset_properties[:,2]
        Y_len = dataset_properties[:,0]

In [ ]:
df = pd.DataFrame(Y_len)
ax = df.plot.hist(bins=100, alpha=0.5)

In [ ]:
graph_type = "vis"
classif = "graph" 
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
#paths
path_main = "datasets/dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask.npz" # path to mask dataset used for random
output = generate_output()

In [ ]:
        k = 7005
        g = NaturalVG(weighted='distance')
        #g = HorizontalVG(weighted='distance')
        
        g.build(X[k])

        adj_mat_vis = np.zeros([len(X[k]),len(X[k])], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_vis[x,y] = q
            adj_mat_vis[y,x] = q
        
        edge_index = torch.from_numpy(adj_mat_vis).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat_vis[row, col]
        plt.imshow(adj_mat_vis)

In [ ]:
len(x.T)

In [ ]:
df=output[5012]

edge_index = pd.DataFrame(df.edge_index.numpy())
x = pd.DataFrame(df.x.numpy())[:20].T

plt.xlabel("Time steps")
plt.ylabel("RSSI")

col =[]

for i in range(len(x.T)):
    if x[i][0]<-80:
        col.append('red')  
    else:
        col.append('blue') 

#plt.plot(x.T,linewidth=2)



for i in range(len(edge_index.T[:78-44])):
    plt.plot(edge_index[i],[x[edge_index[i][0]][0],x[edge_index[i][1]][0]],c='grey',linewidth=0.5)
for i in range(len(x.T)):
    plt.plot(i,x[i].T, marker=".",markersize=15,  c = col[i])
    
plt.savefig('vis_plot.eps', format='eps')
plt.savefig('vis_plot.png', format='png')

In [ ]:
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from matplotlib import pyplot
import matplotlib.pyplot as plt

k=8002

datax = output[k]
datax.x = datax.x#[:20]
datax.edge_index = datax.edge_index#[:,:78-44]
datax.edge_attr = datax.edge_attr#[:78-44]
mask = X_mask[k]#[:20]
color_map = []
for i in range(len(mask)):
    if mask[i] == 0:
        color_map.append('yellow')
    else: 
        color_map.append('red')     
g = to_networkx(data = datax, to_undirected = True)
nx.draw(g, node_size=500, with_labels=True, node_color = color_map)

plt.savefig('graph_plot.eps', format='eps')
plt.savefig('graph_plot.png', format='png')

In [ ]:
output[4011]

In [ ]:
datax.edge_index[:,:78-32]


In [ ]:
# importing the required module
import timeit
 
# code snippet to be executed only once
SETUP_CODE = '''
import pandas as pd
import numpy as np
import ts2vg
from pyts.image import MarkovTransitionField
from ts2vg import NaturalVG
from ts2vg import HorizontalVG
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
 '''

# code snippet whose execution time is to be measured
TEST_CODE = '''
graph_type = "MTF"
classif = "graph" 
len_type = "un/cut" 
vis_type = "natural"
vis_distance = 'distance'
vis_edge_type = "undirected"

path_main = "datasets/dataset_uncut.csv"
        
df = pd.read_csv(path_main)  
del df['Unnamed: 0']
df.index, df.columns = [range(df.index.size), range(df.columns.size)]
length_rss = int((df.columns.stop-2)/2)
global X_mask
X = df.loc[:,df.columns[:length_rss]].to_numpy()
Y = df[length_rss+1].to_numpy(dtype=np.uint8)
X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
if graph_type in ("MTF", "vis_on_MTF", "MTF_on_vis"):
    MTF = MarkovTransitionField(n_bins=length_rss)
    X_gaf_MTF = MTF.fit_transform(X)
output = []

def vis_matrix(X_current, vis_type_local):
    if vis_type_local == "natural":
        g = NaturalVG(weighted=vis_distance)
    elif vis_type_local == "horizontal":
        g = HorizontalVG(weighted=vis_distance)

    g.build(X_current)

    adj_mat_vis = np.zeros([len(X_current),len(X_current)], dtype='float')
    for i in range(len(g.edges)):
        x, y, q =g.edges[i]
        adj_mat_vis[x,y] = q
        if vis_edge_type == "undirected":
            adj_mat_vis[y,x] = q

    return adj_mat_vis

# functions for creating edge index and edge weight for a given matrix
def adjToEdgidx(adj_mat):
    #function for visibility and MTF matrixes
    edge_index = torch.from_numpy(adj_mat).nonzero().t().contiguous()
    row, col = edge_index
    edge_weight = adj_mat[row, col]
    return edge_index,edge_weight

def define_mask(i):
    if classif == "graph": # for graph classification
        return torch.tensor(Y[i], dtype=torch.long)      
    elif classif == "node":# for node classification 
        return torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)

if graph_type == "MTF":  
    for i, j in enumerate(X_gaf_MTF):
        edge_index, edge_weight = adjToEdgidx(j)
        y_mask = define_mask(i)
        #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally label
        output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1), y=y_mask))
elif graph_type == "vis":
    for i in range(len(X)):
        edge_index, edge_weight = adjToEdgidx(vis_matrix(X[i],vis_type))
        y_mask = define_mask(i)
        output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=torch.tensor(edge_index, dtype=torch.int64), edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1),y=y_mask))    
'''
 
# timeit statement
print (timeit.repeat(setup = SETUP_CODE,
                     stmt = TEST_CODE,
                     repeat = 5,
                     number = 1))

In [ ]:
k=2123
d = 4

df0=output[k*0+d]
df1=output[k*1+d]
df2=output[k*2+d]
df3=output[k*3+d]
df4=output[k*4+d]
edge_index = pd.DataFrame(df.edge_index.numpy())
x0 = pd.DataFrame(df0.x.numpy()).T
x1 = pd.DataFrame(df1.x.numpy()).T
x2 = pd.DataFrame(df2.x.numpy()).T
x3 = pd.DataFrame(df3.x.numpy()).T
x4 = pd.DataFrame(df4.x.numpy()).T


plt.xlabel("Time steps")
plt.ylabel("RSSI")

col =[]

# for i in range(len(x1.T)):
#     if x1[i][0]<-80:
#         col.append('red')  
#     else:
#         col.append('blue') 

#plt.plot(x.T,linewidth=2)



# for i in range(len(edge_index.T)):
#     plt.plot(edge_index[i],[x[edge_index[i][0]][0],x[edge_index[i][1]][0]],c='grey',linewidth=0.5)
#for i in range(len(x.T)):
plt.plot(x0.T, marker=".",markersize=1,  c = "red")
plt.plot(x1.T, marker=".",markersize=1,  c = "blue")
plt.plot(x2.T, marker=".",markersize=1,  c = "green")
plt.plot(x3.T, marker=".",markersize=1,  c = "black")
plt.plot(x4.T, marker=".",markersize=1,  c = "yellow")
    
plt.savefig('vis_plot.eps', format='eps')
plt.savefig('vis_plot.png', format='png')

In [ ]:


y = list(range(300)) 
x = df3.x.numpy().T[0]
plt.plot(y, x, '-o')
plt.fill_between(y, x, color='blue', alpha=0.5, where=(y > y.mean()))
plt.show()

In [ ]:
x